In [1]:
import pandas as pd
import pydeck as pdk
import datetime
import time
from ipywidgets import IntSlider, Play, HBox, VBox, HTML, jslink,interactive_output, Layout

### Preprocesamiento del dataset

El dataset consiste en un conjunto de puntos con coordenadas en latitud y longitud en EPSG = 4326, con una estampa de tiempo en UTC. Cada punto representa una posición en un momento dado de un elemento identificado por el agent_id. Los datos cubren dos semanas, entre el 11 y el 25 de agosto.

Antes de pasar los datos a la capa de Deck, es necesario realizar un par de pasos de preprocesamiento:
* Pasar el utc_timestamp a una timestamp legible
* Concatenar latitud y longitud en un arreglo de coordenadas
* Con la finalidad de seleccionar el día con más puntos, se extrae la fecha en una columna separada.

In [2]:
df = pd.read_csv('./data/traj_data.csv')
df['datetime'] = pd.to_datetime(df.utc_timestamp, unit = 's')
df['coordinates'] = df[['longitude', 'latitude']].apply(list, axis = 1)
df['day'] = df.datetime.dt.day

In [3]:
df.day.unique()

array([12, 13, 11, 21, 20, 15, 14, 22, 23, 24, 25, 18, 19, 16, 17])

In [4]:
df

,agent_id,latitude,longitude,utc_timestamp,datetime,coordinates,day
0,1,25.688278,-100.321875,1628755047,2021-08-12 07:57:27,"[-100.321875, 25.688278]",12
1,1,25.688347,-100.321910,1628768277,2021-08-12 11:37:57,"[-100.32191, 25.688347]",12
2,1,25.688387,-100.321768,1628834712,2021-08-13 06:05:12,"[-100.321768, 25.688387]",13
3,1,25.688270,-100.321780,1628774508,2021-08-12 13:21:48,"[-100.32178, 25.68827]",12
4,1,25.688337,-100.321895,1628754000,2021-08-12 07:40:00,"[-100.321895, 25.688337]",12
...,...,...,...,...,...,...,...
1629344,7879,25.697962,-100.287476,1629756590,2021-08-23 22:09:50,"[-100.287476, 25.697962]",23
1629345,7879,25.713190,-100.271860,1629592179,2021-08-22 00:29:39,"[-100.27186, 25.71319]",22
1629346,7879,25.713196,-100.271835,1629592184,2021-08-22 00:29:44,"[-100.271835, 25.713196]",22
1629347,7879,25.713190,-100.271850,1629592170,2021-08-22 00:29:30,"[-100.27185, 25.71319]",22



El performance del equipo presenta un limitante por lo que elegiremos 100 de los agent_ids con más puntos durante el día con más puntos del dataset. Para ello tenemos que el día 13 cuenta con 167745 puntos:

In [5]:
df.day.value_counts()

13    167745
21    155024
12    154339
20    147745
14    142778
22    135100
19    129399
15    129123
16    118849
18    109209
17     99608
23     92429
24     30775
11     10562
25      6664
Name: day, dtype: int64

In [6]:
df = df[df.day == 13]

En promedio tenemos ~3773 puntos por agent_id

In [7]:
sum(df.agent_id.value_counts()[0:100])//100

445

In [8]:
df.agent_id.value_counts()[0:100]

1       1415
1545    1323
788     1224
3916     912
62       892
        ... 
1456     272
2527     271
3421     270
2605     270
720      270
Name: agent_id, Length: 100, dtype: int64

In [9]:
df = df[df.agent_id.isin(df.agent_id.value_counts()[0:100].index)]

Si bien se creó una variable legible del tiempo en los pasos anteriores, se usará el timestamp para la animación debido a que es más fácil realizar los incrementos sobre los valores enteros. Para el rango tenemos:

In [10]:
df.utc_timestamp.min()

1628812800

In [11]:
df.utc_timestamp.max()

1628899199

La capa de trayectoria de deck.gl usa un formato especifico, donde los puntos que forman un "path" deben de estar en un arreglo de las mismas dimensiones que las muestras de tiempo. Siguiendo la documentación [de la capa](https://deckgl.readthedocs.io/en/latest/gallery/trips_layer.html) se creo un dataframe para los 100 agent_ids con esas características

In [12]:
data = []
for aid in df.agent_id.unique():
    temp = df[df.agent_id == aid].sort_values(by = 'utc_timestamp')
    data.append({'agent_id': aid, 'path':temp.coordinates.to_list(), 'time':temp.utc_timestamp.to_list(), 'color':[228, 87, 86]})

In [13]:
df2 = pd.DataFrame.from_dict(data)

In [14]:
df2

,agent_id,path,time,color
0,1,"[[-100.32183, 25.68835], [-100.321813, 25.6883...","[1628813226, 1628813276, 1628813276, 162881327...","[228, 87, 86]"
1,105,"[[-100.071528, 25.648474], [-100.07149, 25.648...","[1628813133, 1628814305, 1628814305, 162881435...","[228, 87, 86]"
2,145,"[[-100.20466, 25.73743], [-100.20468, 25.73743...","[1628812929, 1628812929, 1628812932, 162881293...","[228, 87, 86]"
3,347,"[[-100.25387, 25.79466], [-100.25387, 25.79465...","[1628847852, 1628847852, 1628847852, 162884786...","[228, 87, 86]"
4,601,"[[-100.265293, 25.587966], [-100.26151, 25.589...","[1628814421, 1628814436, 1628814436, 162881444...","[228, 87, 86]"
...,...,...,...,...
95,1925,"[[-100.63622, 23.65195], [-100.63621, 23.65196...","[1628815756, 1628815766, 1628816463, 162881651...","[228, 87, 86]"
96,187,"[[-100.50555, 25.68698], [-100.50556, 25.68697...","[1628812811, 1628812822, 1628812932, 162881294...","[228, 87, 86]"
97,2423,"[[-100.346326, 25.65811], [-100.34628, 25.6580...","[1628813047, 1628813055, 1628813060, 162881310...","[228, 87, 86]"
98,4015,"[[-100.35206, 25.75612], [-100.35205, 25.75611...","[1628812806, 1628812806, 1628812856, 162881285...","[228, 87, 86]"


### Trips layer

El trips layer es una representación gráfica de un viaje. La capa toma el punto inicial y, de acuerdo a un arreglo de unidades temporales ordenado, va dibujando una línea que se desvanece. Entonces, en el momento dado el punto en t está en un color más sólido que los puntos en t-1, t-2.. , t-n. El parametro de esta "cola" se llama trail_length.

La capa puede ser renderizada de manera "estática" (el mapa aún puede moverse) o si se ajusta el parámetro de current time puede animarse.

En el contexto de pydeck (los bindings de deck.gl usados para el proyecto) una visualización está conformado por tres puntos: 
* Un objeto view (o ViewState) que representa la vista inicial del mapa y no puede ser actualizado.
* Una serie de capas. Véase: [este link](https://deckgl.readthedocs.io/en/latest/index.html) para mayor información
* Un objeto deck que junta todos estos. Este objeto puede ser renderizado a html o ejecutado como un widget de jupyter. 

Este último punto es la razón por la cuál el proyecto está siendo entregado por este medio. Las capas dentro de un jupyter widget si pueden ser actualizadas permitiendo animarlas.

In [18]:
view = {"bearing": 0, "latitude": 25.66861111,  "longitude":-100.30972222, "pitch": 0, "zoom": 11}

time_min = df.utc_timestamp.min()-200
time_max = df.utc_timestamp.max()+200

layer = pdk.Layer(
    "TripsLayer",
    df2,
    get_path='path',
    get_timestamps='time',
    get_color='color',
    opacity=0.9,
    width_min_pixels=3,
    rounded=True,
    trail_length=800,
    current_time=0
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view, map_style='road', height = 650)

In [19]:
def getReadableTime(utc):
    return(datetime.datetime.utcfromtimestamp(utc).strftime('%d-%b || %H:%M:%S'))

Para la visualización se usaron 3 widgets de jupyter que proveen de interactividad:
* Un slider que permite ajustar la ventana de tiempo deseada
* Un botón de play que "reproduce" la animación del movimiento de los puntos
* Un cuadro de texto que muestra el tiempo en formato normal. 

In [20]:
slider = IntSlider(value=time_min, min=time_min, max=time_max, step=50)
play = Play(value=time_min, min=time_min, max=time_max, step=250, description='Mostrar viajes', interval=250)
text = HTML(description = "<b>Time</b>", value = getReadableTime(time_min), layout = Layout(width = '100%'))
jslink((play, 'value'), (slider, 'value'))
hbox = HBox([play,slider],layout = Layout(width = '100%'))
layout = VBox([text,hbox])

# function
def update_plot(utc):
    layer.current_time = utc
    text.value = getReadableTime(utc)
    return r.update()
    

# interaction between widget and function
interact = interactive_output(update_plot, {'utc': slider})
display(layout, interact)
r.show()

Output()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, height=650, json_input='{"initialViewS…

### Conclusiones

Este proyecto me sirvió para entender como implementar bajo condiciones limitadas algún tipo de animación. Originalmente pensaba realizar el proyecto en react+javascript donde la librería de deck.gl tiene más funcionalidades. Pero al final por cuestiones de tiempo opté por usar los bindings para python. Esto me limitaba a desarrollar algo dentro del entorno de jupyter pero usando unos trucos con los widgets para crear callbacks y entendiendo la animación como un cambio en términos del tiempo de una serie de puntos pude logra desarrollar la visualización. Me hubiera gustado probarlo en un entorno con GPU para visualizar los 3700 identificadores únicos. Creo que hubiera sido más atractivo visualmente. Trataré de implementarlo en los siguientes días en react esperando mejorar el rendimiento; en esta iteración hay demasiadas capaz de por medio => deck - mapbox - jupyter - python.

También resta la pregunta si este tipo de tareas no se pudiera beneficiar de paralelismo implementado a manera de futuros en python dado que no todos los puntos se deben de dibujar de manera simultanea, podrían usarse hilos y no bloquear el intérprete. 